In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").appName("TeamProject").getOrCreate()

In [4]:
spark

In [103]:
df = spark.read.option("header","true").csv("\\Users\\Mohammad Dayyat\\pyspark\\googleplaystore.csv")

In [6]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [104]:
df=df.dropna()

In [105]:
df.dropDuplicates()

DataFrame[App: string, Category: string, Rating: string, Reviews: string, Size: string, Installs: string, Type: string, Price: string, Content Rating: string, Genres: string, Last Updated: string, Current Ver: string, Android Ver: string]

In [9]:
df.count()

10839

In [107]:
from pyspark.sql.types import StringType , IntegerType ,DoubleType
from pyspark.sql.functions import *
from pyspark.sql.functions import col, cast 

In [11]:
df = df.withColumn("Installs", regexp_replace("Installs", "\+", ""))
df = df.withColumn("Installs", regexp_replace("Installs", ",", ""))
df = df.withColumn("Installs", col("Installs").cast("integer"))

In [ ]:
df = df.withColumn("Price", regexp_replace(col("Price"), "[^\d.]", ""))
df = df.withColumn("Price", col("Price").cast(DoubleType()))
df = df.na.fill(0, "Price")

In [122]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: double (nullable = false)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [18]:
from pyspark.sql.functions import when
from pyspark.sql.functions import sum,avg,max,count

In [110]:
df.createOrReplaceTempView("gstore")

In [31]:
# Top 5 downloaded apps (show to which category they belong to)
top_down = df.select(["Category","App","Installs"]).orderBy('Installs',ascending = False)
top_down.show(5)

+-------------------+--------------------+----------+
|           Category|                 App|  Installs|
+-------------------+--------------------+----------+
|      COMMUNICATION|            Hangouts|1000000000|
|      COMMUNICATION|  WhatsApp Messenger|1000000000|
|      COMMUNICATION|Google Chrome: Fa...|1000000000|
|BOOKS_AND_REFERENCE|   Google Play Books|1000000000|
|      COMMUNICATION|Messenger – Text ...|1000000000|
+-------------------+--------------------+----------+
only showing top 5 rows



In [32]:
spark.sql("SELECT Category , App , Installs FROM gstore ORDER BY Installs desc limit 5").show()

+-------------------+--------------------+----------+
|           Category|                 App|  Installs|
+-------------------+--------------------+----------+
|      COMMUNICATION|               Gmail|1000000000|
|      COMMUNICATION|Google Chrome: Fa...|1000000000|
|BOOKS_AND_REFERENCE|   Google Play Books|1000000000|
|      COMMUNICATION|Messenger – Text ...|1000000000|
|      COMMUNICATION|            Hangouts|1000000000|
+-------------------+--------------------+----------+



In [34]:
spark.sql("SELECT COUNT(App) FROM gstore WHERE Installs =1000000000 ").show()

+----------+
|count(App)|
+----------+
|        58|
+----------+



In [121]:
# Highest paid categories
spark.sql("SELECT Category,SUM(Price) as Total_price FROM gstore  group by Category order by Total_price desc").show(35)

+-------------------+------------------+
|           Category|       Total_price|
+-------------------+------------------+
|            FINANCE| 2900.829999999998|
|             FAMILY| 2434.779999999982|
|          LIFESTYLE|           2360.87|
|            MEDICAL|1439.9600000000007|
|               GAME| 287.3000000000003|
|              TOOLS| 267.2500000000002|
|       PRODUCTIVITY|250.93000000000012|
|           BUSINESS|185.26999999999998|
|    PERSONALIZATION|153.95999999999998|
|        PHOTOGRAPHY|134.20999999999995|
|BOOKS_AND_REFERENCE|119.76999999999997|
|             EVENTS|            109.99|
|             SPORTS| 99.99999999999996|
|      COMMUNICATION| 83.13999999999999|
| HEALTH_AND_FITNESS|             67.34|
|   TRAVEL_AND_LOCAL| 49.95000000000002|
|            WEATHER| 32.42000000000001|
|             DATING|31.430000000000007|
|MAPS_AND_NAVIGATION|26.950000000000003|
|          EDUCATION|             17.96|
|             SOCIAL|             15.97|
|  AUTO_AND_VEHI